In [2]:
import pandas as pd

In [3]:
df = pd.read_json("/home/alex/paper-2025-anonymous-submission/Data/processed_data/ria_dataset.json")    

In [10]:
df["lead"] = df["lead"].apply(lambda x: x.replace(u'\xa0', u' '))

In [11]:
# classification simple

In [12]:
system_propmt = 'Присутствует ли в заголовке новости игра слов? Дай ответ с учетом содержания новости. Отвечать можешь только "да", "нет" или "не знаю".'

In [13]:
def genereate_user_prompt(headline, lead):
    user_prompt = f"Заголовок новости: {headline}. Cодержание новости: {lead}"
    return user_prompt

In [14]:
df["user_prompt"] = df.apply(
    lambda row: genereate_user_prompt(headline=row["headline"], lead=row["lead"]),
    axis=1
)

df["system_prompt"] = system_propmt

In [16]:
df[["user_prompt", "system_prompt"]].to_json(
    "/home/alex/paper-2025-anonymous-submission/Data/processed_data/ria_detection.json",
    orient="index",
    force_ascii=False
)

In [17]:
system_extended_instruction = """
Присутствует ли в заголовке новости игра слов? Дай ответ с учетом содержания новости. Отвечать можешь только "да", "нет" или "не знаю".
Игрой слов считаются следующие механизмы:

1. Фонетический уровень

Механизм: Омофония
Определение: Слова, звучащие одинаково, но имеющие разное написание и значение.
Примеры:
    - С толпы отечества
    - Ремонт жданий
Основные индикаторы: Идентичное произношение при различном написании.

2. Морфологический уровень

Механизм: Словообразование
Определение: Создание новых слов или выражений путём модификации или комбинации существующих.
Примеры:
    - Полунедостаточность
    - Повторение пролётанного
Основные индикаторы: Необычные словесные конструкции, играющие с знакомыми
морфемами или частями слов.

3. Лексико-фразеологический уровень

Механизм: Трансформация фразеологизмов
Определение: Изменение известных фраз, идиом или пословиц для создания нового
смысла или комического эффекта.
Примеры:
    - Приговор дороже ночи
    - На свой страх и иск
Основные индикаторы: Изменения в фразеологизмах, сохраняющие некоторую структуру оригинальной фразы, но с изменёнными словами.

Механизм: Полисемия
Определение: Использование слов с несколькими значениями для создания двусмысленности или юмора.
Примеры:
    - ЦБ пошел по картам
    - Алюминий сплавляют на экспорт
Основные индикаторы: Одно слово в заголовке имеет более одного вероятного толкования.

Механизм: Омонимия
Определение: Слова, одинаковые по написанию и звучанию, но разные по значению.
Примеры:
    - ЛУКОЙЛу сохраняют «Надежду»
    - Следствие зрит в ОКО
Основные индикаторы: Идентичное написание и произношение с различным значением в контексте.

Механизм: Оксюморон
Определение: Сочетание противоположных по значению слов для создания парадоксального эффекта.
Примеры:
    - У МВД попросили показать документики
    - «Нижний» верхнему не соперник
Основные индикаторы: Фразы, содержащие внутренне противоречивые идеи.

Механизм: Трансформация устойчивых сочетаний
Определение: Изменение или перестройка известных устойчивых сочетаний (терминов, выражений) для создания нового смысла или комического эффекта.
Примеры:
    - Главное — не Виктория, а участие
    - К стартапам подошли экосистемно
Основные индикаторы: Частичная замена или перестройка известных устойчивых сочетаний, которая сохраняет распознаваемую основу оригинала, но добавляет новый смысл или комический эффект.

4. Дискурсивный уровень
Механизм: Отсылка к прецедентным текстам
Определение: Ссылка на известные тексты, фильмы, песни или исторические события,
немного изменённые для нового контекста.
Примеры:
    - В наши банки заходили корабли
    - Квад на земле
Основные индикаторы: Знание оригинального текста критично, заголовок часто вызывает коннотации оригинального произведения.

Дополнительно
Трансформация фразеологизма, пословицы, поговорки или прецедентного текста может осуществляться несколькими способами:
- с помощью замены одного из слов другим словом;
- с помощью вставки нового слова в середину устойчивого сочетания;
- с помощью добавления к одному из слов устойчивого сочетания приставки;
- с помощью отсечения приставки (обратный процесс);
- с помощью замены части звуков слова, то есть посредством подбора сходно звучащего слова. Слова могут иметь схожее окончание;
- с помощью усечения конца слов;
- с помощью изменения окончания.
"""

In [18]:
df["system_prompt"] = [system_extended_instruction for _ in range(len(df))]

In [19]:
df[["user_prompt", "system_prompt"]].to_json(
    "/home/alex/paper-2025-anonymous-submission/Data/processed_data/ria_detection_extended.json",
    orient="index",
    force_ascii=False
)

In [20]:
system_instruction_interpretation = f"""
Проанализируй заголовок новости в контексте ее содержания.
Укажи, есть ли в заголовке игра слов. Если она есть, объясни смысл, использованные методы и связь с основным текстом.
Если игры слов нет, то ответь "в заголовке нет игры слов".
"""

In [21]:
df["system_prompt"] = [system_instruction_interpretation for _ in range(len(df))]

In [22]:
df[["user_prompt", "system_prompt"]].to_json(
    "/home/alex/paper-2025-anonymous-submission/Data/processed_data/ria_interpretation.json",
    orient="index",
    force_ascii=False
)

In [26]:
system_extended_instruction_interpretation = """
Проанализируй заголовок новости в контексте ее содержания.
Укажи, есть ли в заголовке игра слов. Если она есть, объясни смысл, использованные методы и связь с основным текстом.
Если игры слов нет, то ответь "в заголовке нет игры слов".
Игрой слов считаются следующие механизмы:

1. Фонетический уровень

Механизм: Омофония
Определение: Слова, звучащие одинаково, но имеющие разное написание и значение.
Примеры:
    - С толпы отечества
    - Ремонт жданий
Основные индикаторы: Идентичное произношение при различном написании.

2. Морфологический уровень

Механизм: Словообразование
Определение: Создание новых слов или выражений путём модификации или комбинации существующих.
Примеры:
    - Полунедостаточность
    - Повторение пролётанного
Основные индикаторы: Необычные словесные конструкции, играющие с знакомыми
морфемами или частями слов.

3. Лексико-фразеологический уровень

Механизм: Трансформация фразеологизмов
Определение: Изменение известных фраз, идиом или пословиц для создания нового
смысла или комического эффекта.
Примеры:
    - Приговор дороже ночи
    - На свой страх и иск
Основные индикаторы: Изменения в фразеологизмах, сохраняющие некоторую структуру оригинальной фразы, но с изменёнными словами.

Механизм: Полисемия
Определение: Использование слов с несколькими значениями для создания двусмысленности или юмора.
Примеры:
    - ЦБ пошел по картам
    - Алюминий сплавляют на экспорт
Основные индикаторы: Одно слово в заголовке имеет более одного вероятного толкования.

Механизм: Омонимия
Определение: Слова, одинаковые по написанию и звучанию, но разные по значению.
Примеры:
    - ЛУКОЙЛу сохраняют «Надежду»
    - Следствие зрит в ОКО
Основные индикаторы: Идентичное написание и произношение с различным значением в контексте.

Механизм: Оксюморон
Определение: Сочетание противоположных по значению слов для создания парадоксального эффекта.
Примеры:
    - У МВД попросили показать документики
    - «Нижний» верхнему не соперник
Основные индикаторы: Фразы, содержащие внутренне противоречивые идеи.

Механизм: Трансформация устойчивых сочетаний
Определение: Изменение или перестройка известных устойчивых сочетаний (терминов, выражений) для создания нового смысла или комического эффекта.
Примеры:
    - Главное — не Виктория, а участие
    - К стартапам подошли экосистемно
Основные индикаторы: Частичная замена или перестройка известных устойчивых сочетаний, которая сохраняет распознаваемую основу оригинала, но добавляет новый смысл или комический эффект.

4. Дискурсивный уровень
Механизм: Отсылка к прецедентным текстам
Определение: Ссылка на известные тексты, фильмы, песни или исторические события,
немного изменённые для нового контекста.
Примеры:
    - В наши банки заходили корабли
    - Квад на земле
Основные индикаторы: Знание оригинального текста критично, заголовок часто вызывает коннотации оригинального произведения.

Дополнительно
Трансформация фразеологизма, пословицы, поговорки или прецедентного текста может осуществляться несколькими способами:
- с помощью замены одного из слов другим словом;
- с помощью вставки нового слова в середину устойчивого сочетания;
- с помощью добавления к одному из слов устойчивого сочетания приставки;
- с помощью отсечения приставки (обратный процесс);
- с помощью замены части звуков слова, то есть посредством подбора сходно звучащего слова. Слова могут иметь схожее окончание;
- с помощью усечения конца слов;
- с помощью изменения окончания.
"""

In [27]:
df["system_prompt"] = [system_extended_instruction_interpretation for _ in range(len(df))]

In [28]:
df[["user_prompt", "system_prompt"]].to_json(
    "/home/alex/paper-2025-anonymous-submission/Data/processed_data/ria_interpretation_extended.json",
    orient="index",
    force_ascii=False
)